#### Comparing Sentinel C-Band SAR to Capella X-Band SAR

In [ ]:
import ee, geemap
ee.Initialize(project='ee-krle4401')

aoi = ee.Geometry.Rectangle([])
capella_date = ee.Date('2025-03-20')  # from your JSON

# Search ± 6 months for HH in any mode
window_days = 180
s1_hh_any = (ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(aoi)
    .filterDate(capella_date.advance(-window_days, 'day'),
                capella_date.advance(window_days, 'day'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
)

# Sort by temporal distance to your Capella date
def add_time_diff(img):
    diff = img.date().difference(capella_date, 'day').abs()
    return img.set('time_diff_days', diff)

s1_hh_sorted = s1_hh_any.map(add_time_diff).sort('time_diff_days')

closest_hh = ee.Image(s1_hh_sorted.first())
print("Closest HH scene date:", closest_hh.date().format().getInfo())

import ee, geemap, numpy as np, rasterio
ee.Initialize(project='ee-krle4401')

# keep your Sentinel HH layer
Map = geemap.Map(center=[31.105, 121.325], zoom=12)
Map.addLayer(closest_hh.select('HH'), {'bands':['HH'], 'min':-25, 'max':0}, 'Closest S1 HH (dB)')
Map.addLayer(aoi, {}, 'AOI')

# --- Local Capella GeoTIFF overlay (no GEE upload) ---
capella_path = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs/shanghai_hh_db.tif"

# Auto-stretch (fast percentiles) for nicer quick-look
with rasterio.open(capella_path) as src:
    arr = src.read(1, masked=True).compressed()
    p2, p98 = np.percentile(arr, [2, 98])  # adjust if needed

Map.add_raster(
    capella_path,
    layer_name="Capella HH (local dB)",
    vmin=float(p2), vmax=float(p98),
    opacity=0.65
)

#Generate an HTML file rather than rendering it in notebook locally, it will make life a lot easier...
Map.to_html("sentinel_capella_compare.html", title="Sentinel-1 vs Capella HH", width="100%", height="800px")

print("Map saved to sentinel_capella_compare.html — open it in your browser.")



EEException: The Geometry.Rectangle constructor requires 2 points or 4 coordinates.

In [15]:
import json, numpy as np
from pyproj import Transformer
import ee, geemap, rasterio
import os

ee.Initialize(project='ee-krle4401')

# --- 1) Load Capella metadata ---
json_path = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/data/raw/scene_shanghai/CAPELLA_C11_SP_GEO_HH_20250320045730_20250320045802_extended.json"
with open(json_path, "r") as f:
    meta = json.load(f)

# Geotransform (UTM 51N, EPSG:32651)
gt = meta["collect"]["image"]["image_geometry"]["geotransform"]
rows = int(meta["collect"]["image"]["rows"])
cols = int(meta["collect"]["image"]["columns"])
x0, px_w, rot_x, y0, rot_y, px_h_neg = gt
px_h = abs(px_h_neg)

# Corners in UTM (no rotation for this product)
width_m  = px_w * cols
height_m = px_h * rows
corners_utm = np.array([
    [x0,             y0            ],  # TL
    [x0 + width_m,   y0            ],  # TR
    [x0 + width_m,   y0 - height_m ],  # BR
    [x0,             y0 - height_m ],  # BL
    [x0,             y0            ],  # close
], dtype=float)

# Reproject to lon/lat
transformer = Transformer.from_crs(32651, 4326, always_xy=True)
lons, lats = transformer.transform(corners_utm[:,0], corners_utm[:,1])
footprint_ll = [[float(lon), float(lat)] for lon, lat in zip(lons, lats)]
centroid_lon = float(np.mean(lons[:-1])); centroid_lat = float(np.mean(lats[:-1]))

# --- 2) Correct center time from JSON ---
# Prefer center_pixel.center_time; fall back to start/stop if needed
center_iso = (meta.get("collect", {})
                  .get("image", {})
                  .get("center_pixel", {})
                  .get("center_time"))
if center_iso is None:
    center_iso = meta["collect"]["start_timestamp"]  # safe fallback

capella_date = ee.Date(center_iso)
print("Capella center time:", center_iso)

# AOI polygon
aoi = ee.Geometry.Polygon([footprint_ll])

# --- 3) Find closest Sentinel scene (HH if available, else VV/VH) ---
window_days = 180
s1_hh = (ee.ImageCollection('COPERNICUS/S1_GRD')
         .filterBounds(aoi)
         .filterDate(capella_date.advance(-window_days,'day'),
                     capella_date.advance(window_days,'day'))
         .filter(ee.Filter.listContains('transmitterReceiverPolarisation','HH')))

def add_time_diff(img):
    return img.set('time_diff_days', img.date().difference(capella_date, 'day').abs())

s1_hh_sorted = s1_hh.map(add_time_diff).sort('time_diff_days')

if s1_hh_sorted.size().getInfo() > 0:
    s1_img  = ee.Image(s1_hh_sorted.first()); s1_band = 'HH'
    s1_vis  = {'bands':[s1_band], 'min':-25, 'max':0}
    print("Closest Sentinel‑1 HH:", s1_img.date().format().getInfo())
else:
    s1_iw = (ee.ImageCollection('COPERNICUS/S1_GRD')
             .filterBounds(aoi)
             .filterDate(capella_date.advance(-window_days,'day'),
                         capella_date.advance(window_days,'day'))
             .filter(ee.Filter.eq('instrumentMode','IW'))
             .sort('system:time_start'))
    s1_img = ee.Image(s1_iw.first())
    bands  = s1_img.bandNames().getInfo()
    s1_band = 'VV' if 'VV' in bands else 'VH'
    s1_vis  = {'bands':[s1_band],
               'min': -22 if s1_band=='VV' else -27,
               'max':   0 if s1_band=='VV' else  -5}
    print("Closest Sentinel‑1 (IW):", s1_img.date().format().getInfo(), "| Band:", s1_band)

# --- 4) Map & local Capella overlay ---
Map = geemap.Map(center=[centroid_lat, centroid_lon], zoom=13)
Map.addLayer(s1_img.select(s1_band), s1_vis, f"S1 {s1_band} (closest)")
Map.addLayer(aoi, {'color':'red'}, "Capella footprint (exact)")

capella_path = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs/shanghai_hh_db.tif"  # adjust if needed
with rasterio.open(capella_path) as src:
    arr = src.read(1, masked=True).compressed()
    p2, p98 = np.percentile(arr, [2, 98])

Map.add_raster(capella_path, layer_name="Capella HH (local dB)",
               vmin=float(p2), vmax=float(p98), opacity=0.65)

Map.split_map(left_layer=f"S1 {s1_band} (closest)", right_layer="Capella HH (local dB)")
project_dir = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs"
os.makedirs(project_dir, exist_ok=True)
out_html = os.path.join(project_dir, "sentinel_capella_compare.html")
Map.to_html(out_html, title="Capella vs Sentinel (aligned)", width="100%", height="820px")
print("Saved: sentinel_capella_compare.html")


Capella center time: 2025-03-20T04:57:46.562000Z
Closest Sentinel‑1 HH: 2025-04-11T09:45:46
Saved: sentinel_capella_compare.html


Even just looing between the two, the difference is staggering. It is worth reminding you that these are different bands.

![Sentinel](https://github.com/kllewers/Capella_Space_Dabbling/blob/main/Sentinel_data.png)
![Both](https://github.com/kllewers/Capella_Space_Dabbling/blob/main/Sentinel_Capella_Data.png)

#### Only look at overlapping pixels in the AOI with that have Data Values